In [1]:
# Import libraries
#!pip install openpyxl --upgrade
import pandas as pd
import re
import ollama
import numpy as np

In [2]:
# Read case summaries Excel file and create a DataFrame
df = pd.read_excel("case_summarization.xlsx")  
df['Body']=df['Body'].apply(str)

df.head()

,Case: Case Number,Case: Subject,Case: Description,Case: Product Name,Automation Item Name,Created Date,Body,Feedback,Detailed Feedback,Case: Status,Case: Client Escalated
0,TS017493063,*JLO* GBM-Kuwait-KOC-ESS- PROACTIVE ACTION AGA...,I am reaching out to discuss the IBM support a...,Elastic Storage System Hardware,AI-1559438,2024-10-13,Customer Sentiment negative Description Summar...,NaN,NaN,IBM is working,0
1,TS017525407,Email - We required high uninitialized Luns/ho...,We required high uninitialized Luns/hosts (Hig...,DS8900F,AI-1560218,2024-10-14,Customer Sentiment Not available Description S...,NaN,good,Closed by IBM,0
2,TS017529903,fca72a160 7/12 -> fca72a163 7/8 QSFP port do n...,fca72a160 7/12 -> fca72a163 7/8 QSFP port do n...,SAN b-type Collection,AI-1560770,2024-10-14,Customer Sentiment neutral Description Summary...,NaN,NaN,Closed by Client,0
3,TS014486964,SSD problem,Please note that there is a problem with one o...,Power System S914 Server,AI-1560820,2024-10-14,Customer Sentiment neutral Description Summary...,NaN,NaN,Awaiting your feedback,0
4,TS017567622,Blade unexpectedly powered off. No entries in ...,Synergy blade ohecp1esxi002 unexpectedly power...,Synergy Series Frames,AI-1561315,2024-10-14,Customer Sentiment negative Description Summar...,NaN,NaN,Closed by IBM,0


In [3]:
from functools import wraps
import shelve
import hashlib

def cached(func):
    func.cache = shelve.open('llm_cache')
    @wraps(func)
    def wrapper(*args):
        h = hashlib.sha512(str(args).encode('utf-8')).hexdigest()
        try:
            return func.cache[h]
        except KeyError:
            func.cache[h] = result = func(*args)
            func.cache.close()
            func.cache = shelve.open('llm_cache')
            return result
    return wrapper

In [ ]:
# Process case summaries with LLM
@cached
def get_RCA_from_LLM(context, prompt, model):
#The RCA must at least contain the root cause, analysis, resolution, and impact.
    response = ollama.chat(
        model=model,
        messages=[{
            'role': 'user',
            'content': prompt + context
        }]
    )
    return response['message']['content']

# Loop through cases and generate RCA
prompt = """
Here is an example of an RCA:
{
  "RCA": {
    "incident_id": "",
    "date": "",
    "problem": {
      "description": "Memory leak causing service degradation in Service C",
      "symptoms": "Service C experienced gradual performance degradation over 3 hours.",
      "detected_by": "Monitoring system - High memory usage alert"
    },
    "root_cause": {
      "description": "Improper memory management in the data processing module.",
      "related_services": ["Service A", "Service B"],
      "historical_pattern": "Similar memory leak issues found in Service A (Jan 2021) and Service B (Mar 2022)."
    },
    "resolution": {
      "description": "Applied memory optimization techniques, including better memory allocation strategies and garbage collection triggers.",
      "steps_taken": [
        "Identified the memory leak using memory profiler tools.",
        "Optimized memory usage in the data processing module.",
        "Tested the fix in a controlled environment."
      ],
      "time_to_resolve": "2 hours",
      "service_downtime": "3 hours",
      "service_restoration": "Full service restored after memory optimization."
    },
    "impact": {
      "description": "Service degradation for 3 hours, impacting 15% of users.",
      "business_impact": "Moderate performance issues, leading to delays in processing large files."
    },
    "preventive_actions": {
      "description": "Implemented proactive memory monitoring and automated garbage collection.",
      "actions": [
        "Added memory usage alerts in the monitoring system.",
        "Conducted code review to improve memory handling in related services."
      ],
      "future_risk": "High likelihood of similar issues in Service D due to similar code structure."
    }
  }
}
Assume the role of an SRE engineer and give me an RCA report based on the following summary.
Return the report in json format.
If you don't have the data or it's unclear return an empty string :\n
"""
RCA = []
model='granite3-dense:8b-instruct-fp16'
for i, row in df.iterrows():
    context = 'ID: '+row['Case: Case Number']+"\n"    
    context += 'Subject: '+row['Case: Subject']+"\n" 
    context += 'Description: '+row['Case: Description']+"\n" 
    context += 'Created Date: '+str(row['Created Date'])+"\n" 
    context += 'Summary: '+row['Body']+"\n" 
    print(context)
    RCA += [get_RCA_from_LLM(context, prompt, model)]
    if i > 5: break


In [15]:
# Print Sample RCA
i = 0
print('Original Summary: ',df['Body'][i])
print('RCA:', RCA[i])

Original Summary:  Customer Sentiment negative Description Summary The customer is concerned about a high failure rate of NVDIMMs in IBM Storage Scale System 5000 with Power9 servers (MTM 5105-22E) running code 6.1.8.3-6.1.9.2. They are seeking a proactive call to discuss measures and an implementation plan. The relevant IBM support article can be found here: . Feed Summary Not available Resolution Summary Confirmed Solution Not Summarized Solution Attempts *Generated by watsonx*: There is a rpc-statd issue on node 'prot3'. The suggested resolution is to reboot 'prot3' using the commands 'mmces node suspend --stop -N ibmessprot3', 'mmshutdown -N ibmessprot3 -f', and 'reboot'. After the reboot, the node should be resumed using 'mmces node resume --start -N ibmessprot3' and the health status should be checked. Additionally, some Ganesha configuration tuning is recommended, such as modifying the 'MDCACHE' and 'NFS\_Core\_Param' blocks in the gpfs.ganesha.main.conf file and restarting the 

In [16]:
# Initialize vector DB in memory
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient(":memory:")

In [17]:
# Create a collection
# Add RCAs to vector database
idx = client.add(
    collection_name="rca_collection",
    documents=RCA
)

In [18]:
# Search RCA dataset

# query = "firewall issue"
query = "a faulty drive was detected"

search_result = client.query(
    collection_name="rca_collection",
    query_text=query
)

# Print TOP 3 results
for i, r in enumerate(search_result):
    print('score:', r.score)
    print(r.metadata['document'])
    if i+1 >= 3: break

score: 0.9092899736736496
{
  "RCA": {
    "incident_id": "TS014486964",
    "date": "2024-10-14",
    "problem": {
      "description": "SSD (2D35-FFF6) detected recoverable error and potential disk drive failure.",
      "symptoms": "Failing function code 722, unknown FRU part number, and disk drive pdisk0/hdisk0 faulty.",
      "detected_by": "Customer service representative (CSR)"
    },
    "root_cause": {
      "description": "Potential disk drive failure in SSD (2D35-FFF6).",
      "related_services": ["rootvg"],
      "historical_pattern": "N/A"
    },
    "resolution": {
      "description": "Replacement of the faulty disk drive.",
      "steps_taken": [
        "Analysis of SNAP file to identify the faulty disk drive.",
        "Removal of the faulty disk from the mirror and volume group prior to replacement."
      ],
      "time_to_resolve": "N/A",
      "service_downtime": "N/A",
      "service_restoration": "N/A"
    },
    "impact": {
      "description": "Potential data